In [1]:
import transformers
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch.nn.functional as F

/home/rtx4090/miniconda3/envs/ADS/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ai_text=[]
with open('llama_ai_it.txt','r') as f:
    line = f.readline()
    while line:
        line = line.strip()
        line = line.replace('start : ',' ')
        line = line.replace('[\"',' ')
        line = line.replace('[\'',' ')
        line = line.replace('\"]',' ')
        line = line.replace('\']',' ')
        ai_text.append(line)
        print(line)
        line = f.readline()

  \n\nBiografia \nNato a Helsinki da una famiglia appartenente alla piccola nobiltà finlandese, il suo padre era un diplomatico che lavorava per il governo finlandese. \n\n### Pagina precedente\n\n\n 
  L\'uomo senza passato (Mies vailla menneisyyttä)\nThe book was published in 1987 and is the first part of a trilogy, the second part being L\'uomo senza memoria (Mies vailla muistoa) and the third part L\'uomo senza anima (Mies vailla hengitystä).\nThe book is about a man who is a cyborg and who has no past, no memories, no soul.\nThe book is set in a post-apocalyptic world where humanity has been reduced to a small group of survivors who live in a city surrounded by a wasteland.\nThe main character, who is referred to as "the man without a past," is a cyborg who has no memories of his past life and is unable to form new ones.\nHe is forced to live in the city, where he is subjected to various forms of torture and abuse, and is eventually killed by the city\'s ruler.\nThe book was writt

In [ ]:
human_it_tr =[]
with open('human_it_tr.txt', 'r') as f:
    line = f.readline()
    while line :
        line  = line.strip()
        human_it_tr.append(line)
        line = f.readline()

In [4]:

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-it-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-it-en")
model.eval()

model.to(torch.device('cuda'))


/home/rtx4090/miniconda3/envs/ADS/lib/python3.11/site-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(80379, 512, padding_idx=80378)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(80379, 512, padding_idx=80378)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [5]:
for i in range(len(ai_text)):
    ai_text[i] = ai_text[i][:1024]

ai_de_tr =[]
count = 0
for text in ai_text:
    inp = text[:512]
    input_ids = tokenizer(inp, return_tensors="pt").input_ids
    input_ids = input_ids.to(torch.device('cuda'))
    outputs = model.generate(input_ids=input_ids, num_beams=5, num_return_sequences=1)
    ai_de_tr.append(tokenizer.batch_decode(outputs, skip_special_tokens=True))
    count+=1
    print(count)
    del input_ids
    torch.cuda.empty_cache()


print('Ai translation done')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [7]:
ai_it_tr = ai_de_tr

In [6]:
with open('llama_ai_it_tr.txt', 'w') as f:
    for line in ai_de_tr:
        f.write(str(line)+'\n')

In [8]:
device = "cuda"# example: cuda:0
detector_path_or_id = "TrustSafeAI/RADAR-Vicuna-7B"
detector = transformers.AutoModelForSequenceClassification.from_pretrained(detector_path_or_id)
detector_tokenizer = transformers.AutoTokenizer.from_pretrained(detector_path_or_id)
detector.eval()
detector.to(device)

Text_input = ai_it_tr
# error = 0
output_probs_list_it_ai_tr=[]
# Use detector to deternine wehther the text_input is ai-generated.
with torch.no_grad():
  for i  in Text_input:
    inputs = detector_tokenizer(i, padding=True, truncation=True, max_length=512, return_tensors="pt")
    inputs = {k:v.to(device) for k,v in inputs.items()}
    output_probs = F.log_softmax(detector(**inputs).logits,-1)[:,0].exp().tolist()
    output_probs_list_it_ai_tr.append(output_probs)
# output_probs is the probability that the input_text is generated by LLM.
# for result in output_probs:
#   error +=  result 
# error = error/len(output_probs)
print(len(output_probs_list_it_ai_tr))
file_ai = "llama_ai_it_tr.csv"

1000


In [9]:
with open(file_ai, 'w') as f:
    for num in output_probs_list_it_ai_tr:
        f.write(str(num[0]))




In [10]:
Text_input = ai_text
# error = 0
output_probs_list_it_ai=[]
# Use detector to deternine wehther the text_input is ai-generated.
with torch.no_grad():
  for i  in Text_input:
    inputs = detector_tokenizer(i, padding=True, truncation=True, max_length=512, return_tensors="pt")
    inputs = {k:v.to(device) for k,v in inputs.items()}
    output_probs = F.log_softmax(detector(**inputs).logits,-1)[:,0].exp().tolist()
    output_probs_list_it_ai.append(output_probs)
# output_probs is the probability that the input_text is generated by LLM.
# for result in output_probs:
#   error +=  result 
# error = error/len(output_probs)
print(len(output_probs_list_it_ai))
file_ai = "llama_ai_it.csv"

1000


In [ ]:
with open(file_ai, 'w') as f:
    for num in output_probs_list_it_ai:
        f.write(str(num[0])+"\n")